In [1]:
import pandas as pd
import numpy as np

ancilary_service = pd.read_csv('/Users/leroy/Documents/GitHub/Electricity_trading/Battery_opt/Data/ancillary_service_2024.csv')

ancilary_service.columns = [s.replace(" ","") for s in ancilary_service.columns]



ancilary_service.iloc[:,3:].mean()

REGDN    3.439229
REGUP    6.007119
RRS      5.674302
NSPIN    6.222699
ECRS     8.231750
dtype: float64

In [2]:
ancilary_service.columns

Index(['DeliveryDate', 'HourEnding', 'RepeatedHourFlag', 'REGDN', 'REGUP',
       'RRS', 'NSPIN', 'ECRS'],
      dtype='object')

In [3]:
DAM_price = pd.read_csv('/Users/leroy/Documents/GitHub/Electricity_trading/DART/Data/2023_24_ERCOT_forecast_actual_data_v3.csv')

DAM_price.DA_Price_North.mean()

49.389340561677926

In [4]:
ancilary_service_one = ancilary_service[ancilary_service['DeliveryDate'] == '01/01/2024'].copy()

print(ancilary_service_one.head())

DAM_one = DAM_price[DAM_price.marketday == '1/1/2024'].copy()

print(DAM_one.head())

  DeliveryDate HourEnding RepeatedHourFlag  REGDN  REGUP   RRS  NSPIN  ECRS
0   01/01/2024      01:00                N   1.51   1.49  1.00   0.94  0.10
1   01/01/2024      02:00                N   1.25   1.47  1.00   0.94  0.11
2   01/01/2024      03:00                N   1.25   1.80  1.00   1.23  0.12
3   01/01/2024      04:00                N   1.25   1.60  1.00   1.23  0.13
4   01/01/2024      05:00                N   1.25   2.00  1.01   1.23  0.14
     marketday  hourending  SP_Price_Houston  SP_Price_North  SP_Price_Panh  \
7871  1/1/2024           1           13.8800         14.8700        15.2750   
7872  1/1/2024           2           15.9775         16.0900        16.4050   
7873  1/1/2024           3           16.9950         16.9950        16.9950   
7874  1/1/2024           4           17.9525         17.9525        17.9525   
7875  1/1/2024           5           20.6575         20.6575        20.6575   

      SP_Price_South  SP_Price_West  DA_Price_Houston  DA_Price_North

In [9]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# Model parameters
T = 24  # Number of hours (e.g., 24 for a day)
P_max = 10  # Maximum power capacity (MW)
E_max = 20  # Maximum energy capacity (MWh)
E_min = 2  # Minimum energy capacity (MWh)
eta_discharge = 0.95  # Discharge efficiency
eta_charge = 0.95  # Charge efficiency
E_initial = 10  # Initial state of charge (MWh)
delta_t = 1  # Time step in hours

# Assume price predictions are provided (example values)
price_rt = DAM_one.SP_Price_Houston.values  # Real-time prices for each hour ($/MWh)
price_da = DAM_one.DA_Price_Houston.values  # Day-ahead prices for each hour ($/MWh)
price_reg_up = ancilary_service_one.REGUP.values  # Reg Up prices ($/MW-h)
price_reg_down = ancilary_service_one.REGDN.values  # Reg Down prices ($/MW-h)
price_rrs = ancilary_service_one.RRS.values  # RRS prices ($/MW-h)
price_ecrs = ancilary_service_one.ECRS.values  # ECRS prices ($/MW-h)

# ==============================
# Create model
# ==============================
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(0, T-1)  # 0..23 if T=24

# ==============================
# Decision variables
# ==============================

# Charge and discharge power (MW)
model.P_discharge = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.P_charge    = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))

# Net real-time power injection (MW), can be negative for consumption
model.P_rt = pyo.Var(model.T, domain=pyo.Reals, bounds=(-P_max, P_max))

# Day-ahead energy commitment (MWh). Bounded below to allow negative if you buy DA.
model.E_da = pyo.Var(model.T, domain=pyo.Reals)

# Ancillary service capacities (MW). 
# Bounded to ensure they do not exceed the battery’s max power in any hour.
model.R_up   = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.R_down = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.R_rrs  = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.R_ecrs = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))

# Battery state of charge (MWh)
model.E_bat = pyo.Var(model.T, domain=pyo.Reals, bounds=(E_min, E_max))

# Fix the initial SOC
model.E_bat[0].fix(E_initial)

# ==============================
# Constraints
# ==============================

# 1) Real-time power definition:
#    P_rt[t] = P_discharge[t] - P_charge[t]
def p_rt_definition(m, t):
    return m.P_rt[t] == m.P_discharge[t] - m.P_charge[t]
model.p_rt_def = pyo.Constraint(model.T, rule=p_rt_definition)

# 2) Energy balance:
#    E_bat[t] = E_bat[t-1] - discharge + charge (with efficiency factors)
def energy_balance(m, t):
    if t == 0:
        return pyo.Constraint.Skip  # Already fixed E_bat[0]
    return m.E_bat[t] == (m.E_bat[t-1]
                          - (m.P_discharge[t] * delta_t) / eta_discharge
                          + (m.P_charge[t]    * delta_t) * eta_charge)
model.energy_balance = pyo.Constraint(model.T, rule=energy_balance)

# 3) Maximum power constraint:
#    Real-time injection + upward reserves cannot exceed P_max
def power_max_constraint(m, t):
    return (m.P_rt[t] + m.R_up[t] + m.R_rrs[t] + m.R_ecrs[t]) <= P_max
model.power_max = pyo.Constraint(model.T, rule=power_max_constraint)

# 4) Minimum power constraint:
#    Real-time injection - downward reserves cannot drop below -P_max
def power_min_constraint(m, t):
    return (m.P_rt[t] - m.R_down[t]) >= -P_max
model.power_min = pyo.Constraint(model.T, rule=power_min_constraint)

# 5) ECRS energy constraint (example/placeholder):
#    If we promise R_ecrs[t] for 2 hours, the battery must hold enough energy
def ecrs_energy_constraint(m, t):
    return m.E_bat[t] >= (m.R_ecrs[t] * 2.0 * delta_t) / eta_discharge
model.ecrs_energy = pyo.Constraint(model.T, rule=ecrs_energy_constraint)

# 6) Bounds on day-ahead commitment to prevent unboundedness:
#    E_da[t] must be within ±(P_max * delta_t)
for t in model.T:
    model.E_da[t].setlb(-P_max * delta_t)
    model.E_da[t].setub(P_max  * delta_t)

# 7) (Optional) Link day-ahead MWh to actual physical discharge:
#    E_da[t] <= (P_discharge[t] * delta_t)
def couple_da_discharge(m, t):
    return m.E_da[t] <= m.P_discharge[t] * delta_t
model.couple_da_discharge = pyo.Constraint(model.T, rule=couple_da_discharge)

# ==============================
# Objective
# ==============================
def obj_rule(m):
    return sum(
        # Real-time energy revenue: (P_rt[t] in MW) × price_rt[t] ($/MWh) × delta_t (h)
        m.P_rt[t] * price_rt[t] * delta_t
        # Day-ahead profit: E_da[t] (MWh) × (price_da[t] - price_rt[t])  
        + m.E_da[t] * (price_da[t] - price_rt[t])
        # Ancillary service capacity payments
        + m.R_up[t]   * price_reg_up[t]
        + m.R_down[t] * price_reg_down[t]
        + m.R_rrs[t]  * price_rrs[t]
        + m.R_ecrs[t] * price_ecrs[t]
        for t in m.T
    )
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# ==============================
# Solve
# ==============================
solver = SolverFactory('glpk', executable='/usr/local/bin/glpsol')
results = solver.solve(model, tee=True)  # tee=True prints solver output

# Check solver status
status = results.solver.status
term_cond = results.solver.termination_condition

if (status != pyo.SolverStatus.ok) or (term_cond != pyo.TerminationCondition.optimal):
    print(f"\nNo feasible (or optimal) solution found.\n"
          f"Solver Status: {status}\n"
          f"Termination Condition: {term_cond}")
else:
    print("\nOptimal solution found.\n")

    # Print results
    for t in model.T:
        print(f"Hour {t}: "
              f"P_rt    = {pyo.value(model.P_rt[t]):6.2f}, "
              f"E_da    = {pyo.value(model.E_da[t]):6.2f}, "
              f"R_up    = {pyo.value(model.R_up[t]):6.2f}, "
              f"R_down  = {pyo.value(model.R_down[t]):6.2f}, "
              f"R_rrs   = {pyo.value(model.R_rrs[t]):6.2f}, "
              f"R_ecrs  = {pyo.value(model.R_ecrs[t]):6.2f}, "
              f"E_bat   = {pyo.value(model.E_bat[t]):6.2f}")

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpdved4_af.glpk.raw
 --wglp /var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmph_thog1k.glpk.glp
 --cpxlp /var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpyaal18u3.pyomo.lp
Reading problem data from '/var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpyaal18u3.pyomo.lp'...
143 rows, 215 columns, 402 non-zeros
1199 lines were read
Writing problem data to '/var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmph_thog1k.glpk.glp'...
1219 lines were written
GLPK Simplex Optimizer 5.0
143 rows, 215 columns, 402 non-zeros
Preprocessing...
142 rows, 214 columns, 400 non-zeros
Scaling...
 A: min|aij| =  9.500e-01  max|aij| =  2.105e+00  ratio =  2.216e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 142
*     0: obj =   1.433875000e+03 inf =   0.000e+00 (122)
*    92: obj =   3.732102990e+03 inf =   3.553e-15 (0)


In [10]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

# ==============================
# Model parameters
# ==============================
T = 24  # Number of hours (e.g., 24 for a single day)
P_max = 10.0   # Maximum power capacity (MW)
E_max = 20.0   # Maximum energy capacity (MWh)
E_min = 2.0    # Minimum energy capacity (MWh)
eta_discharge = 0.95  # Discharge efficiency
eta_charge = 0.95     # Charge efficiency
E_initial = 10.0      # Initial state of charge (MWh)
delta_t = 1.0         # Time step in hours

# Example price arrays: real-time, day-ahead, ancillary. 
# (Make sure these arrays each have length T.)
price_rt = DAM_one.SP_Price_Houston.values      # Real-time prices ($/MWh)
price_da = DAM_one.DA_Price_Houston.values      # Day-ahead prices ($/MWh)
price_reg_up = ancilary_service_one.REGUP.values
price_reg_down = ancilary_service_one.REGDN.values
price_rrs = ancilary_service_one.RRS.values
price_ecrs = ancilary_service_one.ECRS.values

# ==============================
# Create model
# ==============================
model = pyo.ConcreteModel()

# Sets
model.T = pyo.RangeSet(0, T-1)  # 0..23 if T=24

# ==============================
# Decision variables
# ==============================

# Charge and discharge power (MW)
model.P_discharge = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.P_charge    = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))

# Net real-time power injection (MW), can be negative for consumption
model.P_rt = pyo.Var(model.T, domain=pyo.Reals, bounds=(-P_max, P_max))

# Day-ahead energy commitment (MWh). Bounded below to allow negative if you buy DA.
model.E_da = pyo.Var(model.T, domain=pyo.Reals)

# Ancillary service capacities (MW). 
# Bounded to ensure they do not exceed the battery’s max power in any hour.
model.R_up   = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.R_down = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.R_rrs  = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))
model.R_ecrs = pyo.Var(model.T, domain=pyo.NonNegativeReals, bounds=(0, P_max))

# Battery state of charge (MWh)
model.E_bat = pyo.Var(model.T, domain=pyo.Reals, bounds=(E_min, E_max))

# Fix the initial SOC
model.E_bat[0].fix(E_initial)

# ==============================
# Constraints
# ==============================

# 1) Real-time power definition:
def p_rt_definition(m, t):
    return m.P_rt[t] == m.P_discharge[t] - m.P_charge[t]
model.p_rt_def = pyo.Constraint(model.T, rule=p_rt_definition)

# 2) Energy balance:
def energy_balance(m, t):
    if t == 0:
        return pyo.Constraint.Skip  # Already fixed E_bat[0]
    return m.E_bat[t] == (
        m.E_bat[t-1]
        - (m.P_discharge[t] * delta_t) / eta_discharge
        + (m.P_charge[t]    * delta_t) * eta_charge
    )
model.energy_balance = pyo.Constraint(model.T, rule=energy_balance)

# 3) Maximum power constraint:
def power_max_constraint(m, t):
    return m.P_rt[t] + m.R_up[t] + m.R_rrs[t] + m.R_ecrs[t] <= P_max
model.power_max = pyo.Constraint(model.T, rule=power_max_constraint)

# 4) Minimum power constraint:
def power_min_constraint(m, t):
    return m.P_rt[t] - m.R_down[t] >= -P_max
model.power_min = pyo.Constraint(model.T, rule=power_min_constraint)

# 5) ECRS energy constraint:
def ecrs_energy_constraint(m, t):
    return m.E_bat[t] >= (m.R_ecrs[t] * 2.0 * delta_t) / eta_discharge
model.ecrs_energy = pyo.Constraint(model.T, rule=ecrs_energy_constraint)

# 6) Bounds on day-ahead commitment to prevent unboundedness
for t in model.T:
    model.E_da[t].setlb(-P_max * delta_t)
    model.E_da[t].setub(P_max  * delta_t)

# 7) (Optional) Link day-ahead MWh to actual physical discharge
def couple_da_discharge(m, t):
    return m.E_da[t] <= m.P_discharge[t] * delta_t
model.couple_da_discharge = pyo.Constraint(model.T, rule=couple_da_discharge)

# ==============================
# Objective
# ==============================
def obj_rule(m):
    return sum(
        m.P_rt[t] * price_rt[t] * delta_t
        + m.E_da[t] * (price_da[t] - price_rt[t])
        + m.R_up[t]   * price_reg_up[t]
        + m.R_down[t] * price_reg_down[t]
        + m.R_rrs[t]  * price_rrs[t]
        + m.R_ecrs[t] * price_ecrs[t]
        for t in m.T
    )
model.obj = pyo.Objective(rule=obj_rule, sense=pyo.maximize)

# ==============================
# Solve
# ==============================
solver = SolverFactory('glpk', executable='/usr/local/bin/glpsol')
results = solver.solve(model, tee=True)

# Check solver status
status = results.solver.status
term_cond = results.solver.termination_condition

if (status != pyo.SolverStatus.ok) or (term_cond != pyo.TerminationCondition.optimal):
    print(f"\nNo feasible (or optimal) solution found.\n"
          f"Solver Status: {status}\n"
          f"Termination Condition: {term_cond}")
else:
    print("\nOptimal solution found.\n")

    # Print results hour-by-hour
    for t in model.T:
        print(f"Hour {t:2d}: "
              f"P_rt    = {pyo.value(model.P_rt[t]):6.2f}, "
              f"E_da    = {pyo.value(model.E_da[t]):6.2f}, "
              f"R_up    = {pyo.value(model.R_up[t]):6.2f}, "
              f"R_down  = {pyo.value(model.R_down[t]):6.2f}, "
              f"R_rrs   = {pyo.value(model.R_rrs[t]):6.2f}, "
              f"R_ecrs  = {pyo.value(model.R_ecrs[t]):6.2f}, "
              f"E_bat   = {pyo.value(model.E_bat[t]):6.2f}")

    # Finally, print the total 24-hour profit
    total_profit = pyo.value(model.obj)
    print(f"\nTotal 24-hour profit = ${total_profit:,.2f}")


GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write /var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmplt7xslx3.glpk.raw
 --wglp /var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpu8802xdu.glpk.glp
 --cpxlp /var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpayynmn91.pyomo.lp
Reading problem data from '/var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpayynmn91.pyomo.lp'...
143 rows, 215 columns, 402 non-zeros
1199 lines were read
Writing problem data to '/var/folders/0r/nzg1qk0n1qd552y9hhd30dnc0000gn/T/tmpu8802xdu.glpk.glp'...
1219 lines were written
GLPK Simplex Optimizer 5.0
143 rows, 215 columns, 402 non-zeros
Preprocessing...
142 rows, 214 columns, 400 non-zeros
Scaling...
 A: min|aij| =  9.500e-01  max|aij| =  2.105e+00  ratio =  2.216e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 142
*     0: obj =   1.433875000e+03 inf =   0.000e+00 (122)
*    92: obj =   3.732102990e+03 inf =   3.553e-15 (0)
